In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seml
import pandas as pd
import json
from collections import defaultdict

/nfs/homedirs/fuchsgru/miniconda3/lib/python3.9/site-packages/seml/database.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
collection_name = 'week5_density_feature_space_big_grid'
collection = seml.database.get_collection(collection_name)
results = [{'config' : r['config'], 'result' : r['result'], 'id' : r['_id']} for r in collection.find() if r['status'] in ('COMPLETED',)]

In [3]:
len(results)

24

# Get best AUROCs for Density Based and Softmax Entropy

In [376]:
def flatten_dict(d):
    new = dict()
    for k, v in d.items():
        if isinstance(v, dict):
            for kk, vv in flatten_dict(v).items():
                new[k + '.' + kk] = vv
        else:
            new[k] = v
    return new
    

In [377]:


names = {
    'model.use_spectral_norm' : 'Spectral Norm',
    'model.residual' : 'Residual',
    'model.weight_scale' : 'Weight Scale',
    'data.train_labels_remove_other' : 'Remove OOD',
    'model.hidden_sizes' : 'Hidden',
}

config_keys = names.keys()


num_best = 5
data = defaultdict(dict)
for result in results:
    cnfg = flatten_dict(result['config'])
    k = tuple(tuple(cnfg[cnfg_key]) if isinstance(cnfg[cnfg_key], list) else float(cnfg[cnfg_key]) for cnfg_key in config_keys)
    # k = tuple(str(x) for x in k)
    if k in data:
        raise RuntimeError(f'{k} is duplicate')
    with open(result['result']) as f:
        metrics = json.load(f)
        grid_auroc = []
        for metric, values in metrics.items():
            values = np.array(values)
            #data[k][metric] = f'{np.array(values).mean():.3f} ± {np.array(values).std():.3f}'
            if 'accuracy' in metric or 'auroc_softmax_entropy' in metric: # Those we want as is
                if metric.startswith('val'): metric = metric[4:]
                #data[tuple(list(k) + ['mean'])][metric] = np.array(values).mean()
                #data[tuple(list(k) + ['std'])][metric] = np.array(values).std()
                data[tuple(list(k))][metric.replace('_', ' ')] = f'{values.mean():.2f} ± {values.std():.2f}'
            elif 'auroc' in metric:
                grid_auroc.append(values)
        grid_auroc = sorted(grid_auroc, key=lambda vals: -vals.mean())
        for idx, values in enumerate(grid_auroc[:num_best]):
            metric = f'{idx + 1}{"st" if idx == 0 else "nd"}-auroc'
            # data[tuple(list(k))][metric] = {'mean' : values.mean(), 'std' : values.std()}
            
            data[tuple(list(k))][metric] = f'{values.mean():.2f} ± {values.std():.2f}'
            #data[tuple(list(k) + ['mean'])][f'{idx + 1}{"st" if idx == 0 else "nd"}-auroc'] = values['mean']
            #data[tuple(list(k) + ['std'])][f'{idx + 1}{"st" if idx == 0 else "nd"}-auroc'] = values['std']
            
    

In [378]:
df = pd.DataFrame(data,)
df.columns.set_names([names[k] for k in config_keys], inplace=True)
df = df.T

In [379]:
df= df.sort_index()

In [383]:
df.iloc[df.index.get_level_values('Remove OOD') == 0].sort_index()

accuracy-val-train-labels  \
Spectral Norm Residual Weight Scale Remove OOD Hidden                               
0.0           0.0      2.0          0.0        (64,)                  0.81 ± 0.04   
                                               (64, 32)               0.82 ± 0.04   
                                               (64, 64)               0.81 ± 0.04   
              1.0      0.9          0.0        (64,)                  0.79 ± 0.04   
                                               (64, 32)               0.81 ± 0.04   
                                               (64, 64)               0.81 ± 0.04   
1.0           0.0      2.0          0.0        (64,)                  0.81 ± 0.03   
                                               (64, 32)               0.83 ± 0.03   
                                               (64, 64)               0.83 ± 0.03   
              1.0      0.9          0.0        (64,)                  0.78 ± 0.04   
                                               (64, 32)               0.80 ± 0.04   
                                               (64, 64)               0.79 ± 0.04   

                                                        accuracy-val-reduced  \
Spectral Norm Residual Weight Scale Remove OOD Hidden                          
0.0           0.0      2.0          0.0        (64,)             0.81 ± 0.04   
                                               (64, 32)          0.82 ± 0.04   
                                               (64, 64)          0.81 ± 0.04   
              1.0      0.9          0.0        (64,)             0.79 ± 0.04   
                                               (64, 32)          0.81 ± 0.04   
                                               (64, 64)          0.81 ± 0.04   
1.0           0.0      2.0          0.0        (64,)             0.81 ± 0.03   
                                               (64, 32)          0.83 ± 0.03   
                                               (64, 64)          0.83 ± 0.03   
              1.0      0.9          0.0        (64,)             0.78 ± 0.04   
                                               (64, 32)          0.80 ± 0.04   
                                               (64, 64)          0.79 ± 0.04   

                                                        auroc softmax entropy  \
Spectral Norm Residual Weight Scale Remove OOD Hidden                           
0.0           0.0      2.0          0.0        (64,)              0.80 ± 0.09   
                                               (64, 32)           0.81 ± 0.05   
                                               (64, 64)           0.81 ± 0.04   
              1.0      0.9          0.0        (64,)              0.83 ± 0.03   
                                               (64, 32)           0.82 ± 0.04   
                                               (64, 64)           0.82 ± 0.02   
1.0           0.0      2.0          0.0        (64,)              0.69 ± 0.07   
                                               (64, 32)           0.75 ± 0.07   
                                               (64, 64)           0.76 ± 0.06   
              1.0      0.9          0.0        (64,)              0.71 ± 0.08   
                                               (64, 32)           0.74 ± 0.08   
                                               (64, 64)           0.78 ± 0.07   

                                                           1st-auroc  \
Spectral Norm Residual Weight Scale Remove OOD Hidden                  
0.0           0.0      2.0          0.0        (64,)     0.72 ± 0.05   
                                               (64, 32)  0.79 ± 0.06   
                                               (64, 64)  0.80 ± 0.07   
              1.0      0.9          0.0        (64,)     0.75 ± 0.10   
                                               (64, 32)  0.75 ± 0.08   
                                               (64, 64)  0.75 ± 0.08   
1.0           0.0      2.0       

# Evaluate the Density Models

In [320]:


names = {
    'model.use_spectral_norm' : 'Spectral Norm',
    'model.residual' : 'Residual',
    'model.weight_scale' : 'Weight Scale',
    'data.train_labels_remove_other' : 'Remove OOD',
    'model.hidden_sizes' : 'Hidden',
}

config_keys = names.keys()


data = []
for result in results:
    cnfg = flatten_dict(result['config'])
    with open(result['result']) as f:
        metrics = json.load(f)
        grid_auroc = []
        for metric, values in metrics.items():
            d = {
                v : cnfg[k] for k, v in names.items()
            }
            values = np.array(values)
            if 'accuracy' in metric or 'auroc_softmax_entropy' in metric: 
                continue
            elif 'auroc' in metric:
                _, density, dim_red = metric.split(':')
                data.append({**d,  **{'mean' : values.mean(), 'std' : values.std(), 'Dimensionality Reduction' : dim_red, 'Density' : density}})
                
                
                

                

In [352]:
def find_best(g):
    return g.loc[g['mean'].idxmax()]

df = pd.DataFrame(data).groupby(['Density', 'Dimensionality Reduction']).apply(find_best).sort_values('mean', ascending=False)


In [370]:
df.head(20)

Spectral Norm  Residual  Weight Scale  \
Density  Dimensionality Reduction                                          
gpc-full 28-isomap                          True      True           0.9   
7-mog    24-isomap                          True      True           0.9   
gpc-diag no                                 True     False           2.0   
gpc-full 32-isomap                          True      True           0.9   
2-mog    32-isomap                          True      True           0.9   
8-mog    16-isomap                          True      True           0.9   
gpc-diag 32-isomap                          True      True           0.9   
         28-isomap                          True      True           0.9   
5-mog    8-isomap                           True      True           0.9   
9-mog    32-isomap                          True      True           0.9   
13-mog   32-isomap                          True      True           0.9   
8-mog    24-isomap                          True      True           0.9   
         32-isomap                          True      True           0.9   
2-mog    28-isomap                          True      True           0.9   
3-mog    20-isomap                          True      True           0.9   
5-mog    24-isomap                          True      True           0.9   
6-mog    20-isomap                          True      True           0.9   
9-mog    20-isomap                          True      True           0.9   
gpc-full 24-isomap                          True      True           0.9   
7-mog    20-isomap                          True      True           0.9   

                                   Remove OOD    Hidden      mean       std  \
Density  Dimensionality Reduction                                             
gpc-full 28-isomap                      False  [64, 32]  0.813750  0.026433   
7-mog    24-isomap                      False  [64, 32]  0.813681  0.022404   
gpc-diag no                             False      [64]  0.813030  0.069184   
gpc-full 32-isomap                      False  [64, 32]  0.811042  0.019525   
2-mog    32-isomap                      False  [64, 32]  0.810903  0.027597   
8-mog    16-isomap                      False  [64, 32]  0.809931  0.025317   
gpc-diag 32-isomap                      False  [64, 32]  0.809236  0.032519   
         28-isomap                      False  [64, 32]  0.809167  0.031060   
5-mog    8-isomap                       False  [64, 32]  0.808194  0.026241   
9-mog    32-isomap                      False  [64, 32]  0.807986  0.021545   
13-mog   32-isomap                      False  [64, 32]  0.807500  0.024417   
8-mog    24-isomap                      False  [64, 32]  0.807292  0.026709   
         32-isomap                      False  [64, 32]  0.807153  0.023213   
2-mog    28-isomap                      False  [64, 32]  0.806389  0.019737   
3-mog    20-isomap                      False  [64, 32]  0.806250  0.029443   
5-mog    24-isomap                      False  [64, 32]  0.805972  0.029715   
6-mog    20-isomap                       True  [64, 64]  0.805903  0.083452   
9-mog    20-isomap                       True  [64, 64]  0.805833  0.084249   
gpc-full 24-isomap                      False  [64, 32]  0.805694  0.022303   
7-mog    20-isomap                       True  [64, 64]  0.804514  0.081320   

                                  Dimensionality Reduction   Density  
Density  Dimensionality Reduction                                     
gpc-full 28-isomap                               28-isomap  gpc-full  
7-mog    24-isomap                               24-isomap     7-mog  
gpc-diag no                                             no  gpc-diag  
gpc-full 32-isomap                               32-isomap  gpc-full  
2-mog    32-isomap                               32-isomap     2-mog  
8-mog    16-isomap                               16-isomap     8-mog  
gpc-diag 32-isomap                               32-iso

In [369]:
df[(df['Spectral Norm'] == True) & (df['Residual'] == False)].head(10)

Spectral Norm  Residual  Weight Scale  \
Density  Dimensionality Reduction                                          
gpc-diag no                                 True     False           2.0   
gpc-full 24-pca                             True     False           2.0   
         28-pca                             True     False           2.0   
         32-pca                             True     False           2.0   
2-mog    no                                 True     False           2.0   
gpc-full 20-pca                             True     False           2.0   
15-mog   16-pca                             True     False           2.0   
16-mog   12-pca                             True     False           2.0   
32-mog   10-pca                             True     False           2.0   

                                   Remove OOD    Hidden      mean       std  \
Density  Dimensionality Reduction                                             
gpc-diag no                             False      [64]  0.813030  0.069184   
gpc-full 24-pca                         False  [64, 64]  0.718021  0.091054   
         28-pca                         False  [64, 64]  0.717240  0.097264   
         32-pca                         False  [64, 64]  0.713125  0.091460   
2-mog    no                             False  [64, 64]  0.610313  0.039168   
gpc-full 20-pca                         False  [64, 64]  0.590573  0.126277   
15-mog   16-pca                         False  [64, 64]  0.586823  0.153610   
16-mog   12-pca                         False  [64, 64]  0.556823  0.106587   
32-mog   10-pca                         False  [64, 64]  0.506354  0.125318   

                                  Dimensionality Reduction   Density  
Density  Dimensionality Reduction                                     
gpc-diag no                                             no  gpc-diag  
gpc-full 24-pca                                     24-pca  gpc-full  
         28-pca                                     28-pca  gpc-full  
         32-pca                                     32-pca  gpc-full  
2-mog    no                                             no     2-mog  
gpc-full 20-pca                                     20-pca  gpc-full  
15-mog   16-pca                                     16-pca    15-mog  
16-mog   12-pca                                     12-pca    16-mog  
32-mog   10-pca                                     10-pca    32-mog

In [372]:
df[df['Dimensionality Reduction'] == 'no'].head(10)

,,Spectral Norm,Residual,Weight Scale,Remove OOD,Hidden,mean,std,Dimensionality Reduction,Density
Density,Dimensionality Reduction,,,,,,,,,
gpc-diag,no,True,False,2.0,False,[64],0.813030,0.069184,no,gpc-diag
32-mog,no,False,True,0.9,True,[64],0.735463,0.069882,no,32-mog
gpc-full,no,False,True,0.9,False,"[64, 64]",0.733780,0.065767,no,gpc-full
6-mog,no,False,True,0.9,False,"[64, 64]",0.733571,0.065127,no,6-mog
16-mog,no,False,True,0.9,True,"[64, 64]",0.730542,0.088275,no,16-mog
8-mog,no,False,True,0.9,False,"[64, 64]",0.719792,0.066985,no,8-mog
9-mog,no,False,True,0.9,True,"[64, 64]",0.713542,0.075728,no,9-mog
20-mog,no,False,True,0.9,True,"[64, 64]",0.711250,0.100990,no,20-mog
7-mog,no,False,True,0.9,False,"[64, 64]",0.710238,0.076721,no,7-mog


In [ ]:
df[df['Remove'] == 'no'].head(10)